### To Do
- Remove unique columns
- Scaling / Normalization
- outlier analysis => 1) scipy import stats (for standardizing values into normal distribution), calc zscore and set threshold 2) IQR
- sklearn imputer - replace missing with mean/median/most frequent/constant OR
- replace missing/garbage by using model

# 1. Removing Unique Columns

#### Progress
- Checking which column has as many unique values in it as the total number of rows. Then dropping the column. Eg: Dropping ID columns.

#### Issues
- If a column has an empty value, the number of unique values decreases and is not equal to number of rows. Hence it is not dropped.
- In case a column isnt actually unique identifier eg: sales in different months. There maybe no same sales in two months. Therefore that important column will be dropped.

#### Solution
- Try to find correlation between target variable and predictor variables. If it is close to 0 then drop it.

In [1]:
import pandas as pd
import glob

In [2]:
def CheckUnique(data):
    col_names = data.columns.to_list()
    print("Original Column Names")
    print(col_names)
    for i in col_names:
        new_col = col_names.copy()
        if data[i].count() == len(data[i].unique()):
            isUniq = True
        else:
            isUniq = False
        if isUniq:
            print("Dropping", i)
            new_col.remove(i)
        return new_col

In [3]:
data = pd.read_csv("./Datasets/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
for i in glob.glob("./Datasets/*"):
    data = pd.read_csv(i)
    print("---------------------------------------------")
    print(i)
    col_names = CheckUnique(data)
    print(col_names)
    print("---------------------------------------------")

---------------------------------------------
./Datasets\bikebuyer1.csv
Original Column Names
['ID', 'Marital Status', 'Gender', 'Yearly Income', 'Children', 'Education', 'Occupation', 'Home Owner', 'Cars', 'Commute Distance', 'Region', 'Age', 'Bike Buyer']
['ID', 'Marital Status', 'Gender', 'Yearly Income', 'Children', 'Education', 'Occupation', 'Home Owner', 'Cars', 'Commute Distance', 'Region', 'Age', 'Bike Buyer']
---------------------------------------------
---------------------------------------------
./Datasets\Boston.csv
Original Column Names
['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat', 'medv']
['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat', 'medv']
---------------------------------------------
---------------------------------------------
./Datasets\cancer.csv
Original Column Names
['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smo

In [5]:
data = pd.read_csv("./Datasets/bikebuyer1.csv")
data

,ID,Marital Status,Gender,Yearly Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Bike Buyer
0,22711.0,Single,Male,30000,0.0,Partial College,Clerical,No,1,1.0,Europe,33,Yes
1,13555.0,Married,Female,40000,0.0,Graduate Degree,Clerical,Yes,0,1.0,Europe,37,Yes
2,NaN,Married,Male,160000,5.0,Partial College,Professional,No,3,2.0,Europe,55,No
3,2.0,Single,Male,160000,0.0,Graduate Degree,Management,Yes,2,5.0,Pacific,47,No
4,25410.0,NaN,Female,70000,2.0,Bachelors,Skilled Manual,No,1,1.0,North America,38,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,22820.0,Married,Male,100000,4.0,High School,Professional,Yes,3,1.0,Europe,52,No
6993,22821.0,Married,Female,130000,4.0,Partial College,Professional,Yes,4,2.0,Europe,52,No
6994,22823.0,Married,Female,160000,5.0,Bachelors,Management,Yes,2,1.0,Europe,53,No
6995,22825.0,Single,Female,120000,5.0,Partial College,Professional,Yes,3,1.0,Europe,54,No


In [6]:
data.describe()

,ID,Yearly Income,Children,Cars,Commute Distance,Age
count,6996.000000,6997.000000,6979.000000,6997.000000,6968.000000,6997.000000
mean,17744.435249,57020.151493,1.111621,1.586823,4.210103,45.107332
std,4337.428859,32080.449720,1.600912,1.146782,2.926211,11.916654
min,2.000000,0.000000,0.000000,0.000000,1.000000,25.000000
25%,14249.750000,30000.000000,0.000000,1.000000,1.000000,36.000000
50%,17406.500000,60000.000000,0.000000,2.000000,4.000000,44.000000
75%,20609.500000,70000.000000,2.000000,2.000000,6.000000,53.000000
max,29476.000000,170000.000000,5.000000,4.000000,13.000000,96.000000


In [7]:
print(data['Yearly Income'])

0        30000
1        40000
2       160000
3       160000
4        70000
         ...  
6992    100000
6993    130000
6994    160000
6995    120000
6996    130000
Name: Yearly Income, Length: 6997, dtype: int64


In [8]:
len(data.count())
data['Yearly Income'].count()

6997

In [9]:
data.corr()

,ID,Yearly Income,Children,Cars,Commute Distance,Age
ID,1.000000,-0.006802,0.012046,0.068315,0.011511,0.021135
Yearly Income,-0.006802,1.000000,0.474575,0.472089,0.009467,0.195351
Children,0.012046,0.474575,1.000000,0.448379,0.026377,-0.000520
Cars,0.068315,0.472089,0.448379,1.000000,0.018397,0.141093
Commute Distance,0.011511,0.009467,0.026377,0.018397,1.000000,-0.026119
Age,0.021135,0.195351,-0.000520,0.141093,-0.026119,1.000000


# 2. Scaling / Normalization

In [10]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6997 entries, 0 to 6996
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                6996 non-null   float64
 1   Marital Status    6981 non-null   object 
 2   Gender            6968 non-null   object 
 3   Yearly Income     6997 non-null   int64  
 4   Children          6979 non-null   float64
 5   Education         6997 non-null   object 
 6   Occupation        6997 non-null   object 
 7   Home Owner        6997 non-null   object 
 8   Cars              6997 non-null   int64  
 9   Commute Distance  6968 non-null   float64
 10  Region            6997 non-null   object 
 11  Age               6997 non-null   int64  
 12  Bike Buyer        6997 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 710.8+ KB


In [12]:
def Scaling(data):
    sc = RobustScaler()
    df = data.copy()
    col_names = data.columns.to_list()
    final_col_names = col_names.copy()
    for i in col_names:
        if str(data[i].dtypes) == "object":
            final_col_names.remove(i)
    features = data[final_col_names]
    df[final_col_names] = sc.fit_transform(features.values)
    return df

In [13]:
data

,ID,Marital Status,Gender,Yearly Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Bike Buyer
0,22711.0,Single,Male,30000,0.0,Partial College,Clerical,No,1,1.0,Europe,33,Yes
1,13555.0,Married,Female,40000,0.0,Graduate Degree,Clerical,Yes,0,1.0,Europe,37,Yes
2,NaN,Married,Male,160000,5.0,Partial College,Professional,No,3,2.0,Europe,55,No
3,2.0,Single,Male,160000,0.0,Graduate Degree,Management,Yes,2,5.0,Pacific,47,No
4,25410.0,NaN,Female,70000,2.0,Bachelors,Skilled Manual,No,1,1.0,North America,38,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,22820.0,Married,Male,100000,4.0,High School,Professional,Yes,3,1.0,Europe,52,No
6993,22821.0,Married,Female,130000,4.0,Partial College,Professional,Yes,4,2.0,Europe,52,No
6994,22823.0,Married,Female,160000,5.0,Bachelors,Management,Yes,2,1.0,Europe,53,No
6995,22825.0,Single,Female,120000,5.0,Partial College,Professional,Yes,3,1.0,Europe,54,No


In [14]:
df = Scaling(data)
df

,ID,Marital Status,Gender,Yearly Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Bike Buyer
0,0.834074,Single,Male,-0.75,0.0,Partial College,Clerical,No,-1.0,-0.6,Europe,-0.647059,Yes
1,-0.605606,Married,Female,-0.50,0.0,Graduate Degree,Clerical,Yes,-2.0,-0.6,Europe,-0.411765,Yes
2,NaN,Married,Male,2.50,2.5,Partial College,Professional,No,1.0,-0.4,Europe,0.647059,No
3,-2.736664,Single,Male,2.50,0.0,Graduate Degree,Management,Yes,0.0,0.2,Pacific,0.176471,No
4,1.258461,NaN,Female,0.25,1.0,Bachelors,Skilled Manual,No,-1.0,-0.6,North America,-0.352941,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6992,0.851213,Married,Male,1.00,2.0,High School,Professional,Yes,1.0,-0.6,Europe,0.470588,No
6993,0.851370,Married,Female,1.75,2.0,Partial College,Professional,Yes,2.0,-0.4,Europe,0.470588,No
6994,0.851684,Married,Female,2.50,2.5,Bachelors,Management,Yes,0.0,-0.6,Europe,0.529412,No
6995,0.851999,Single,Female,1.50,2.5,Partial College,Professional,Yes,1.0,-0.6,Europe,0.588235,No


# 3. Outlier Analysis

# 4. Data Cleaning (replacing values)